<a href="https://colab.research.google.com/github/ebayo/Mining-the-Social-Web-2nd-Edition/blob/master/DPROT_Lab1_Practical_Block_Ciphers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab Work 1: Practical Block Ciphers

## 3 A (somewhat) practical attack against RC4

In this section we propose a easy-to-implement key-recovery attack against a particular mode of operation of RC4. We need several assumptions to launch a successful attack:

* The initialization value iv is a 3-byte counter prepended to a 13-byte long-term key, making a 16-byte string that is used as the actual RC4 key.
* The iv is incremented in every new encryption.
* RC4 is used in a communication system to send some well-structured packets with a constant header of at least one byte m[0].
* The attacker has access to many encryptions, so that he can wait for the use of some specific iv values.

The previous assumptions can be fulfiled in some systems implementing handshake packets encrypted with RC4.

Practical work:

1. From the description of the key scheduling phase of RC4, show that using iv=01 FF x results with high probability in a keystream first byte equal to x+2 , for any byte x .

2. Use it to recover the first (fixed) byte of the encrypted message m[0] . This would require a trial-and-error process, testing all possible values of m[0] and computing how often the first keystream value is equal to . Then, the most likely value for m[0] is the one with the highest frequency.

3. Show now that the first keystream byte produced with iv=03 FF x is, with a noticeable probability, x+6+k[0] , where k[0] denotes the first byte of the long-term key. Similarly, iv=04 FF x produces the first keystream byte equal to x+10+k[0]+k[1] , and so on.

4. Put the previous ideas in practice to learn the first two bytes of the 
long-term key. To do that, generate a 13-byte random key and a one-byte random message (the message header suffices to perform the attack). Then, use the previous special values of iv to generate the encrypted byte for all 256 values of x , and use frequencies to detect the most probable value for m[0] , then k[0] and finally k[1] .

You can use openssl to generate random values (e.g., a 13-byte random string in hexadecimal notation) with

    $ openssl rand -hex 13

Tools like `sort` or `unique` can be useful to detect the most repeated value.

Moreover, you can save all encrypted bytes in a text file and process it with standard tools (spreadsheets) of a script in your preferred programming language (perl, python, C).

```
key is 7e1a0bbc8c770667be44dce10c and message is 90
Gathering keystream first bytes for IV=01FFxx ... done
Guessing m[0] ... done
   Guessed m[0]=90 (with freq.  31)             *** OK ***
Gathering keystream first bytes for IV=03FFxx ... done
Guessing k[0] ... done
   Guessed k[0]=7e (with freq.  16)             *** OK ***
Gathering keystream first bytes for IV=04FFxx ... done
Guessing k[1] ... done
   Guessed k[1]=1a (with freq.  20)             *** OK ***
Gathering keystream first bytes for IV=05FFxx ... done
Guessing k[2] ... done
   Guessed k[2]=0b (with freq.  17)             *** OK ***
Gathering keystream first bytes for IV=06FFxx ... done
Guessing k[3] ... done
   Guessed k[3]=bc (with freq.  13)             *** OK ***
Gathering keystream first bytes for IV=07FFxx ... done
Guessing k[4] ... done
   Guessed k[4]=8c (with freq.  19)             *** OK ***
Gathering keystream first bytes for IV=08FFxx ... done
Guessing k[5] ... done
   Guessed k[5]=77 (with freq.  10)             *** OK ***
Gathering keystream first bytes for IV=09FFxx ... done
Guessing k[6] ... done
   Guessed k[6]=06 (with freq.  12)             *** OK ***
Gathering keystream first bytes for IV=0AFFxx ... done
Guessing k[7] ... done
   Guessed k[7]=67 (with freq.   7)             *** OK ***
Gathering keystream first bytes for IV=0BFFxx ... done
Guessing k[8] ... done
   Guessed k[8]=be (with freq.  12)             *** OK ***
Gathering keystream first bytes for IV=0CFFxx ... done
Guessing k[9] ... done
   Guessed k[9]=44 (with freq.  13)             *** OK ***
Gathering keystream first bytes for IV=0DFFxx ... done
Guessing k[10] ... done
   Guessed k[10]=dc (with freq.   7)             *** OK ***
Gathering keystream first bytes for IV=0EFFxx ... done
Guessing k[11] ... done
   Guessed k[11]=e1 (with freq.  13)             *** OK ***
Gathering keystream first bytes for IV=0FFFxx ... done
Guessing k[12] ... done
   Guessed k[12]=0c (with freq.  10)             *** OK ***

```





### 3.0 Data Preparation

Generate a random header and a random key:

* Random "header", 1 byte: `openssl rand -hex 1 > m0.txt` --> I got the byte 0xbe, but when encrypting openssl sees them as characters, so the actual encrypted message is the character 'b' (0x62)

* Random Key of 13 bytes: `openssl rand -hex 13 > key.txt`


In [ ]:
#example to read one line

fKey = open("key.txt",'r')
KEY = fKey.readline()
print('The 13 byte random key is: {}'.format(KEY))
fM0 = open("m0.txt",'r')
M0_st = fM0.readline()
M0_hex = hex(ord(M0_st))
print('The 1 byte random message is: {} ({})'.format(M0_st,M0_hex))

The 13 byte random key is: a69ae31544d461001888cb2915

The 1 byte random message is: b (0x62)


With the generated key, append the 3-byte pre-pended counter to the key in the range 01FF00 to 01FFFF to generate all possible keys to use, save it to a file to encrypt the message (m0) to be able to do the frontal attack.

In [ ]:
iv = '01ff0'
iv_counter = []
for i in range (0,int('10',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

iv = '01ff'
for i in range (int('10',16),int('100',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

In [ ]:
fKeys = open('keys01ffxx.txt','w')
for iv in iv_counter:
  fKeys.write(iv + KEY)
fKeys.close()

With the file with all the keys with the pre-appended "iv" --> encrypt the header message using openssl

Next we are preparing the output from xxd to keep just the first byte of the encrypeted message

In [ ]:
fcyphers = open('cyphers01ffxx.txt')
cc = fcyphers.readlines()
fcyphers.close()
cyphers = []
for c in cc:
  cyphers.append(c[10:12])

### 3.1 Use of iv = 01 FF xx
From the description of the key scheduling phase of RC4, show that using iv=01 FF x results with high probability in a keystream first byte equal to x+2 , for any byte x .

In [ ]:
#TODO

### 3.2  Recover the first byte of the encrypted message m[0]
Use it to recover the first (fixed) byte of the encrypted message m[0] . This would require a trial-and-error process, testing all possible values of m[0] and computing how often the first keystream value is equal to . Then, the most likely value for m[0] is the one with the highest frequency.

In [ ]:
import numpy as np

In [ ]:
counter = np.zeros(256)
for iv in range(256):
  c = int(cyphers[iv],16)
  for m in range(256):
    if (c^m) == iv+2:
      counter[m] += 1
m_guessed = np.argmax(counter)
freq = max(counter)
print("The encrypted message is {}".format(M0_hex))
print("The guessed message is: {} with prequency {}/{}".format(hex(m_guessed),freq,256*256))

The encrypted message is 0x62
The guessed message is: 0x62 with prequency 33.0/65536


### 3.3 Use of iv = 03 FF xx and iv = 04 FF xx

Show now that the first keystream byte produced with iv=03 FF x is, with a noticeable probability, x+6+k[0] , where k[0] denotes the first byte of the long-term key. Similarly, iv=04 FF x produces the first keystream byte equal to x+10+k[0]+k[1] , and so on.

In [ ]:
# TODO

### 3.4 Recover m[0], k[1], k[2]
Put the previous ideas in practice to learn the first two bytes of the long-term key. To do that, generate a 13-byte random key and a one-byte random message (the message header suffices to perform the attack). Then, use the previous special values of iv to generate the encrypted byte for all 256 values of x , and use frequencies to detect the most probable value for m[0] , then k[0] and finally k[1] .

#### New data preparation

We use the same method as before to obtain all the keys with the preappended data as before with the new values for the counter and obtain the new cyphered message.

In [ ]:
iv = '03ff0'
iv_counter = []
for i in range (0,int('10',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

iv = '03ff'
for i in range (int('10',16),int('100',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

In [ ]:
fKeys = open('keys03ffxx.txt','w')
for iv in iv_counter:
  fKeys.write(iv + KEY)
fKeys.close()

In [ ]:
iv = '04ff0'
iv_counter = []
for i in range (0,int('10',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

iv = '04ff'
for i in range (int('10',16),int('100',16)):
  i_h = hex(i)
  iv_counter.append(iv+i_h[2:])

In [ ]:
fKeys = open('keys04ffxx.txt','w')
for iv in iv_counter:
  fKeys.write(iv + KEY)
fKeys.close()

#### Guessing the first byte of the long term key k[0]

In [ ]:
# Load the encrypted message with iv = 03 ff xx
fcyphers = open('cyphers03ffxx.txt')
cc = fcyphers.readlines()
fcyphers.close()
cyphers = []
for c in cc:
  cyphers.append(c[10:12])

In [ ]:
# x+6+k[0]
counter = np.zeros(256)
for iv in range(256):
  c = int(cyphers[iv],16)
  for k0 in range(256):
    if (c^m_guessed) == (iv+6+k0)%256:
      counter[k0] += 1
k0_guessed = np.argmax(counter)
freq = max(counter)
print("The Key is {}".format(KEY))
print("The guessed message is: {} with prequency {}/{}".format(hex(k0_guessed),freq,256*256))

The Key is a69ae31544d461001888cb2915

The guessed message is: 0xa6 with prequency 17.0/65536


#### Guessing the second byte of the long term key k[1]

In [ ]:
# Load the encrypted message with iv = 04 ff xx
fcyphers = open('cyphers04ffxx.txt')
cc = fcyphers.readlines()
fcyphers.close()
cyphers = []
for c in cc:
  cyphers.append(c[10:12])

In [ ]:
# x+10+k[0]+k[1]
counter = np.zeros(256)
for iv in range(256):
  c = int(cyphers[iv],16)
  for k1 in range(256):
    if (c^m_guessed) == (iv+10+k0_guessed+k1)%256:
      counter[k1] += 1
k1_guessed = np.argmax(counter)
freq = max(counter)
print("The Key is {}".format(KEY))
print("The guessed message is: {} with prequency {}/{}".format(hex(k1_guessed),freq,256*256))

The Key is a69ae31544d461001888cb2915

The guessed message is: 0x9a with prequency 14.0/65536


### 3.5 Some Conclusions

We have seen that knowing some simple information on the production of the key and some information about the message itself, it's easy enough to recover the value of the key and with that RC4 is not really safe. The assumption are not that crazy, speccially with recards to the message as nowadays a lot of information is kept in messages and files with known headers such as internet packages.